In [1]:
import math
import numpy as np
from scipy.stats import norm
import pandas as pd
import yfinance as yf
from datetime import datetime

from dateutil.relativedelta import relativedelta

In [17]:
def bse(stock):
    # Get end date as November 15, 2021
    edate = datetime(2021, 11, 16)

    #Start date as 6 months ago from today
    sdate = edate + relativedelta(months=-6)

    #Change dates to strings
    sdate = datetime.strftime(sdate,"%Y-%m-%d")
    edate = datetime.strftime(edate,"%Y-%m-%d")

    #get historical data of stock
    df = yf.download(stock, start = sdate, end = edate)
    
    #Calculate Log Return
    log_returns = np.log(df['Close']/df['Close'].shift(1))
    
    #Calculate volatility
    sigma = log_returns.std() * math.sqrt(252)#len(df['Close'])

    #Find Risk-Free Rate 12 week t bill 
    r = yf.download('^IRX', start = edate, end = edate)
    r = r['Close'][0]/100

    #Find days until option maturity date
    tick = yf.Ticker(stock)
    print(tick.options[3])
    #t = datetime.strptime(tick.options[3],"%Y-%m-%d") - datetime.strptime(edate,"%Y-%m-%d")
    t = 27/252

    #strikes
    k = [295, 300, 305, 310, 315, 320, 325, 330, 335, 340]

    #stock price
    s = df['Close'][-1]

    print(s,k,sigma,r,t)

    #Black-Scholes Equation
    prices = []
    for x in range(0,len(k)):
        d1 = (np.log(s/k[x])+(r+(sigma**2)/2)*t)/(sigma*np.sqrt(t))
        
        d2 = (np.log(s/k[x])+(r-(sigma**2)/2)*t)/(sigma*np.sqrt(t))
        
        call_price = s*norm.cdf(d1)-k[x]*np.exp(-r*t)*norm.cdf(d2)
        prices.append(call_price)
    
    #Create dataframe of strikes and prices
    z = {'Strike':k, 'Call Price':prices}
    prices = pd.DataFrame(data=z)
    print(prices)

In [18]:
bse('CRM')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2021-12-23
305.489990234375 [295, 300, 305, 310, 315, 320, 325, 330, 335, 340] 0.22428467183119555 0.00039999999105930326 0.10714285714285714
   Strike  Call Price
0     295   15.021935
1     300   11.886288
2     305    9.191631
3     310    6.940512
4     315    5.114094
5     320    3.675726
6     325    2.576430
7     330    1.761056
8     335    1.173953
9     340    0.763394
